In [41]:
import joblib

# Modeli kaydetme
model_filename = r"..\Model\random_forest_model.joblib"

# Modeli yükleme
loaded_model = joblib.load(model_filename)
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


In [70]:
import joblib
import librosa
import numpy as np

# Scaler ve model dosyalarını yükleme
scaler_filename = '..\\Model\\scaler.joblib'  # Modelin scaler dosyasının yolu
model_filename = '..\\Model\\random_forest_model.joblib'  # Model dosyasının yolu

scaler = joblib.load(scaler_filename)
model = joblib.load(model_filename)

def extract_features_for_prediction(file_path):
    """Ses dosyasından özellikleri çıkar."""
    try:
        # Ses dosyasını yükle
        y, sr = librosa.load(file_path, sr=None)
        
        # MFCC (Mel-frequency cepstral coefficients) çıkarımı
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        # Chroma çıkarımı (müzikal tonlar)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=y)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, rms_mean, zcr_mean))
        return features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def predict_audio_with_scaler(file_path):
    """Ses dosyasını scaler ve model ile test et."""
    # Özellik çıkarma
    features = extract_features_for_prediction(file_path)
    if features is None:
        print("Özellik çıkarma başarısız.")
        return

    # Özellikleri normalize et
    features = features.reshape(1, -1)  # Modelin tahmin yapabilmesi için 2D array haline getir
    normalized_features = scaler.transform(features)

    # Model ile tahmin yap
    prediction = model.predict(normalized_features)
    print(f"Tahmin edilen etiket: {prediction[0]}")

# Test edilecek ses dosyası
test_file = "C:\\VoiceProject\\Voice\\irem\\korku\\Iremkorku3.wav"  # Test ses dosyasının tam yolunu yazın
predict_audio_with_scaler(test_file)


Tahmin edilen etiket: irem


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
from flask import Flask, request, jsonify, render_template
import librosa
import numpy as np
import joblib
import os
import logging
from sklearn.preprocessing import StandardScaler

app = Flask(__name__, template_folder='../Frontend')

# Modeli yükleyin (eğitilmiş modelinizin yolunu belirtin)
model = joblib.load("../Model/random_forest_model.joblib")

# Scaler'ı yükleyin (eğitilmiş scaler modelinin yolunu belirtin)
scaler = joblib.load("../Model/scaler_model.joblib")

# Özellik çıkarma fonksiyonu
def extract_features_from_audio(audio_data, sr=16000):
    try:
        # MFCC çıkarımı
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)  # MFCC ortalamasını al

        # Chroma çıkarımı
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=audio_data)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=audio_data)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, rms_mean, zcr_mean))
        return features
    
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None

@app.route('/')
def index():
    # Ana sayfayı (HTML formunu) kullanıcıya göster
    return render_template('index.html')

logging.basicConfig(level=logging.DEBUG)

@app.route('/predict', methods=['POST'])
def predict():
    # Ses dosyasını alın
    audio_file = request.files['file']
    
    logging.debug(f"Dosya alındı: {audio_file.filename}")
    
    # Geçici bir dosyaya kaydedin
    temp_file_path = os.path.join('uploads', audio_file.filename)
    try:
        audio_file.save(temp_file_path)
        logging.debug(f"Dosya kaydedildi: {temp_file_path}")
    except Exception as e:
        logging.error(f"Dosya kaydedilemedi: {e}")
        return jsonify({"error": f"Dosya kaydedilemedi: {e}"}), 500

    try:
        # Ses dosyasını yükleyin
        audio_data, sr = librosa.load(temp_file_path, sr=None)
        logging.debug(f"Ses dosyası başarıyla yüklendi. Örnekleme hızı: {sr}Hz")
    except Exception as e:
        logging.error(f"Ses dosyası yüklenemedi: {e}")
        return jsonify({"error": f"Ses dosyası yüklenemedi: {e}"}), 500

    # 1. Adım: Ses verisini 0-1 arasına normalize etme
    audio_data = (audio_data - np.min(audio_data)) / (np.max(audio_data) - np.min(audio_data))  # 0-1 arası

    # 2. Adım: Özellik çıkarımı
    features = extract_features_from_audio(audio_data)
    if features is None:
        return jsonify({"error": "Ses dosyası işlenemedi."}), 500

    logging.debug(f"Özellikler çıkarıldı: {features}")

    # 3. Adım: Özellikleri normalize etme
    try:
        normalized_features = scaler.transform([features])
        logging.debug(f"Normalize edilmiş özellikler: {normalized_features}")
    except Exception as e:
        logging.error(f"Özellikler normalleştirilemedi: {e}")
        return jsonify({"error": f"Özellikler normalleştirilemedi: {e}"}), 500

    # 4. Adım: Model ile tahmin yapma
    try:
        prediction = model.predict(normalized_features)
        logging.debug(f"Tahmin yapıldı: {prediction}")
        return jsonify({"prediction": prediction[0]})
    except Exception as e:
        logging.error(f"Tahmin yapılamadı: {e}")
        return jsonify({"error": f"Tahmin yapılamadı: {e}"}), 500

if __name__ == '__main__':
    app.run(debug=True)


Ses kaydediliyor...
Ses kaydı tamamlandı.
Normalized Features: [[ 1.4385326   0.41547256  0.14244706  0.58976865  0.60860644  1.06585869
   0.81974664  0.7084436   0.88331646  0.78442708  0.46620559  0.70392766
   0.85344995  0.52587329  0.31204448  0.23396043  0.18220832  0.0877071
   0.0553659   0.05551352  0.07892574  0.16027521  0.5353755   1.18515853
   0.69448006  4.81318403 -0.07128268]]
Tahmin edilen sınıf: irem


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


: 